In [3]:
# Standard libraries
import sys
# Add your custom path

gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import evaluate
from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2

In [7]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 


mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

daily_aggregated_tensors2 = [] 
daily_hourly_maps2 = []      
for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data_keep_ori(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float # or just omit it 
    )
    day_aggregated_tensor[:,0], day_aggregated_tensor[:,1] = day_aggregated_tensor[:,4] , day_aggregated_tensor[:,5]


    for map in day_hourly_map.values():
        # This single line performs the swap safely and efficiently
        map[:,0], map[:,1], map[:,4], map[:,5] = map[:,4], map[:,5], map[:,0], map[:,1]
        
    daily_aggregated_tensors2.append( day_aggregated_tensor )
    daily_hourly_maps2.append( day_hourly_map )
                                
############################## 




tmp_lat = np.concatenate([
    -np.arange(0.176, 2.3, 0.044 * 5)[::-1],
    [-0.132, -0.044, 0, 0.044, 0.132],
    np.arange(0.176, 2.3, 0.044 * 5)
])


lat_deltas = [ ( round(a,3),0 ) for a in tmp_lat]
tolerance = 0.01

instance_sem = evaluate.CrossVariogram(config.mac_save_computed_semi_path,7)
analysis_data_map = daily_hourly_maps2[0]
days_list =[0]
lat_lag_sem = instance_sem.compute_directional_semivariogram(lat_deltas, analysis_data_map, days_list, tolerance)


Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


In [10]:
1/0.063

15.873015873015873

In [8]:
lat_lag_sem 

{0: [(-2.156, 0),
  (-1.936, 0),
  (-1.716, 0),
  (-1.496, 0),
  (-1.276, 0),
  (-1.056, 0),
  (-0.836, 0),
  (-0.616, 0),
  (-0.396, 0),
  (-0.176, 0),
  (-0.132, 0),
  (-0.044, 0),
  (0.0, 0),
  (0.044, 0),
  (0.132, 0),
  (0.176, 0),
  (0.396, 0),
  (0.616, 0),
  (0.836, 0),
  (1.056, 0),
  (1.276, 0),
  (1.496, 0),
  (1.716, 0),
  (1.936, 0),
  (2.156, 0)],
 1: [[17.341299057006836,
   16.90690040588379,
   15.85890007019043,
   15.958700180053711,
   13.360799789428711,
   14.791299819946289,
   11.941399574279785,
   11.039799690246582,
   9.706600189208984,
   7.026899814605713,
   6.146699905395508,
   3.3926000595092773,
   0.0,
   3.3926000595092773,
   6.146699905395508,
   7.026899814605713,
   9.706600189208984,
   11.039799690246582,
   11.941399574279785,
   14.791299819946289,
   13.360799789428711,
   15.958700180053711,
   15.85890007019043,
   16.90690040588379,
   17.341299057006836],
  [26.881500244140625,
   29.25659942626953,
   26.225900650024414,
   21.72500038